<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/code/EDA_%EA%B3%BC%EC%A0%95_0_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
pip install konlpy

In [12]:
import pickle
import random
from konlpy.tag import Okt

# 동의어 사전 로드
def load_synonym_dict(pickle_file):
    with open(pickle_file, 'rb') as f:
        synonym_dict = pickle.load(f)
    return synonym_dict

# Random Swap (랜덤 단어 교환)
def random_swap(words, n=1):
    words = words.copy()
    for _ in range(n):
        idx1, idx2 = random.sample(range(len(words)), 2)
        words[idx1], words[idx2] = words[idx2], words[idx1]
    return words

# Random Insertion (랜덤 단어 삽입)
def random_insertion(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words.insert(idx, random.choice(synonym_dict[word]))
    return words

# Random Delete (랜덤 단어 삭제)
def random_delete(words, p=0.1):
    if len(words) == 1:
        return words
    return [word for word in words if random.random() > p]

# Synonym Replacement (동의어 대체)
def synonym_replacement(words, synonym_dict, n=1):
    words = words.copy()
    for _ in range(n):
        idx = random.randint(0, len(words) - 1)
        word = words[idx]
        if word in synonym_dict:
            words[idx] = random.choice(synonym_dict[word])
    return words

# 데이터 증강 실행 함수
#def augment_text(text, synonym_dict, swap_n=1, insert_n=1, delete_p=0.1, replace_n=1):
#    okt = Okt()
#    words = okt.morphs(text)  # Okt를 이용한 토큰화
#    words = random_swap(words, swap_n)
#    words = random_insertion(words, synonym_dict, insert_n)
#    words = random_delete(words, delete_p)
#    words = synonym_replacement(words, synonym_dict, replace_n)
#    return ' '.join(words)

In [13]:
# 동의어 사전 로드
synonym_dict = load_synonym_dict('/content/wordnet.pickle')

In [15]:
# 데이터 증강 실행 함수
def augment_text(text, synonym_dict, swap_n=0, insert_n=0, delete_p=0, replace_n=0):
    okt = Okt()
    words = okt.morphs(text)  # Okt를 이용한 토큰화

    if swap_n > 0:
        words = random_swap(words, swap_n)
    if insert_n > 0:
        words = random_insertion(words, synonym_dict, insert_n)
    if delete_p > 0:
        words = random_delete(words, delete_p)
    if replace_n > 0:
        words = synonym_replacement(words, synonym_dict, replace_n)

    return ' '.join(words)

# 데이터불러오기

In [16]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'  '요약문증강 최종_0.02_1000개.csv'   이전
'My Drive'	   '요약문증강 최종_0.1_1000개.csv'    졸업논문


In [18]:
import pandas as pd
df= pd.read_csv('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_0.5_증분대상_2500개.csv', encoding='utf-8-sig')

In [20]:
df.head(3)

,id,document,label,길이
0,7652286,솔직히 이영화때문에 슈퍼맨시리즈가 7년이란 세월동안 묻혀있엇다 맨오브스틸은 미국에서...,0,138
1,8895340,아무리 짱짱맨 옵대장의 다때려부수는 CG가 좋다그래도 이딴 디워랑 견줄만한 스토리로...,0,101
2,6549317,오프닝장면 죠스영화 고대루 베겼네?? 여자 비명 연기하는 것도 고대루네~! 그냥 갖...,0,103


In [21]:
# 증강 적용
df['text_swap'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, swap_n=2))
df['text_insert'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, insert_n=2))
df['text_delete'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, delete_p=0.2))
df['text_replace'] = df['document'].apply(lambda x: augment_text(x, synonym_dict, replace_n=2))

In [22]:
df.to_csv('NSMC_0.8_aug_eda.csv', encoding='utf-8-sig')